# Full Data Calculation for Weapons

## Setting Up

In [1]:
import math
import pandas as pd
from scripts.weapon import max_shots, burst, penetrating_damage, hits_to_kill, chance_to_damage
from scripts.accuracy import average_to_hit

aliens = pd.read_csv("datasets/aliens.csv")
weapon_stats = pd.read_csv("datasets/weapon_stats.csv")
ammo_stats = pd.read_csv("datasets/ammo_stats.csv")
weapon_firemodes = pd.read_csv("datasets/weapon_firemodes.csv")
damage_modifiers = pd.read_csv("datasets/damage_modifiers.csv")

## Data Merge

In [2]:
weapon_data = weapon_stats.merge(ammo_stats, on="weapon")
weapon_data = weapon_data.merge(weapon_firemodes, on="weapon")

## Chance to Hit

We will use an average Accuracy of 50. Meaning the soldier fails half of the time.

The chance of hit formula is simple. Just multiply the soldier's accuracy chance by the weapon's accuracy chance. So an average of 50 to a weapon of Accuracy 90 means 0.50 * 0.9 = 0.45 chance of hit.

In [3]:
weapon_data["average_chance_to_hit"] = weapon_data.apply(lambda x: average_to_hit(x["accuracy"]), axis=1)

## Shots

In [4]:
weapon_data["burst"] = weapon_data.apply(burst, axis=1)
weapon_data["turn_max_shots"] = weapon_data.apply(max_shots, axis=1)
weapon_data["turn_max_shots"] = weapon_data.apply(max_shots, axis=1)
weapon_data["turns_to_reload"] = weapon_data.apply(lambda x: x["capacity"] / x["turn_max_shots"], axis=1)
weapon_data['turns_to_reload'] = weapon_data['turns_to_reload'].fillna(0)
weapon_data["turns_to_reload"] = weapon_data.apply(lambda x: x["turns_to_reload"], axis=1)

## Damage Range

In [5]:
weapon_data["turn_damage_max"] = weapon_data.apply(lambda x: x["turn_max_shots"] * x["damage"], axis=1)
weapon_data["turn_damage_expected"] = weapon_data.apply(lambda x: x["turn_damage_max"] * x["average_chance_to_hit"], axis=1)
weapon_data["clip_damage"] = weapon_data.apply(lambda x: x["capacity"] * x["damage"], axis=1)

## Costs

In [6]:
weapon_data["damage_per_time_unit"] = weapon_data.apply(lambda x: x["damage"] / x["time_units"], axis=1)

## Damage to Target Type

### Data Merge

In [7]:
weapon_data_targets = weapon_data.append([weapon_data] * (len(damage_modifiers.index)-1), ignore_index=True)
weapon_data_targets["key"] = weapon_data_targets.groupby(["weapon", "ammo", "fire_mode"]).cumcount()

targets = damage_modifiers["target"]

weapon_data_targets = pd.merge(weapon_data_targets, targets, left_on="key", right_index=True, how="left", sort=False)
weapon_data_targets = weapon_data_targets.drop(columns=["key"])

### Damage vs Type

In [8]:
def damage_modifier(group):
    """
    Returns the damage modifier for the weapon target.
    """
    target = damage_modifiers[damage_modifiers["target"] == group["target"]]
    damage = target[group["damage_type"]].values[0]
    return damage / 100

In [9]:
weapon_data_targets["target_damage"] = weapon_data_targets.apply(lambda x: x["damage"] * damage_modifier(x), axis=1)
weapon_data_targets["turn_target_damage_max"] = weapon_data_targets.apply(lambda x: x["turn_max_shots"] * x["target_damage"], axis=1)
weapon_data_targets["turn_target_damage_expected"] = weapon_data_targets.apply(lambda x: x["turn_target_damage_max"] * x["average_chance_to_hit"], axis=1)

### Costs

In [10]:
weapon_data_targets["target_damage_per_time_unit"] = weapon_data_targets.apply(lambda x: x["target_damage"] / x["time_units"], axis=1)

## Damage to Alien

### Average Armor

In [11]:
aliens_armor = aliens[["alien", "armor_front", "armor_side", "armor_rear", "armor_under"]]
aliens_armor = aliens_armor.set_index("alien")
aliens_armor = aliens_armor.mean(axis=1)
aliens_armor = aliens_armor.reset_index()
aliens_armor = aliens_armor.rename(columns={0: "armor"})

### Base data

In [12]:
weapon_data_aliens = aliens.merge(aliens_armor, on="alien")
weapon_data_aliens = weapon_data_aliens.merge(weapon_data_targets, on="target")
weapon_data_aliens = weapon_data_aliens.rename(columns={"time_units_x": "time_units"})
weapon_data_aliens = weapon_data_aliens.rename(columns={"time_units_y": "weapon_time_units"})

### Penetrating Damage

In [13]:
weapon_data_aliens["chance_to_penetrate"] = weapon_data_aliens.apply(lambda x: chance_to_damage(x["target_damage"], x["armor"]), axis=1)
weapon_data_aliens["chance_of_hit_and_penetrate"] = weapon_data_aliens.apply(lambda x: x["average_chance_to_hit"] * x["chance_to_penetrate"], axis=1)

In [14]:
weapon_data_aliens["penetrating_damage"] = weapon_data_aliens.apply(lambda x: penetrating_damage(x["target_damage"],x["armor"]), axis=1)
weapon_data_aliens["turn_penetrating_damage_max"] = weapon_data_aliens.apply(lambda x: x["turn_max_shots"] * x["penetrating_damage"], axis=1)
weapon_data_aliens["turn_penetrating_damage_expected"] = weapon_data_aliens.apply(lambda x: x["turn_penetrating_damage_max"] * x["average_chance_to_hit"], axis=1)
weapon_data_aliens["clip_penetrating_damage"] = weapon_data_aliens.apply(lambda x: x["capacity"] * x["penetrating_damage"], axis=1)

### Hits to Kill

In [15]:
weapon_data_aliens["hits_to_kill"] = weapon_data_aliens.apply(lambda x: hits_to_kill(x["penetrating_damage"], x["health"]), axis=1)
weapon_data_aliens["turn_expected_kills"] = weapon_data_aliens.apply(lambda x: x["turn_max_shots"] / x["hits_to_kill"], axis=1)

In [16]:
weapon_data_aliens["overkill"] = weapon_data_aliens.apply(lambda x: (x["hits_to_kill"] * x["penetrating_damage"]) - x["health"], axis=1)

### Costs

In [17]:
weapon_data_aliens["penetrating_damage_per_time_unit"] = weapon_data_aliens.apply(lambda x: x["penetrating_damage"] / x["weapon_time_units"], axis=1)
weapon_data_aliens["overkill_per_time_unit"] = weapon_data_aliens.apply(lambda x: x["overkill"] / x["weapon_time_units"], axis=1)

## Data Generated

In [18]:
weapon_data[weapon_data["weapon"]=="heavy_plasma"]

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,burst,turn_max_shots,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit
44,heavy_plasma,2,heavy_plasma_clip,115,plasma,35.0,auto,35,50,0.2750,3,6,5.833333,690,189.7500,4025.0,3.285714
45,heavy_plasma,2,heavy_plasma_clip,115,plasma,35.0,snapshot,30,75,0.4125,1,3,11.666667,345,142.3125,4025.0,3.833333
46,heavy_plasma,2,heavy_plasma_clip,115,plasma,35.0,aimed,60,110,0.6050,1,1,35.000000,115,69.5750,4025.0,1.916667


In [19]:
weapon_data.head()

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,burst,turn_max_shots,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit
0,pistol,1,pistol_clip,26,armor_piercing,12.0,snapshot,18,60,0.3300,1,5,2.400000,130,42.900,312.0,1.444444
1,pistol,1,pistol_clip,26,armor_piercing,12.0,aimed,30,78,0.4290,1,3,4.000000,78,33.462,312.0,0.866667
2,rifle,2,rifle_clip,30,armor_piercing,20.0,auto,35,35,0.1925,3,6,3.333333,180,34.650,600.0,0.857143
3,rifle,2,rifle_clip,30,armor_piercing,20.0,snapshot,25,60,0.3300,1,4,5.000000,120,39.600,600.0,1.200000
4,rifle,2,rifle_clip,30,armor_piercing,20.0,aimed,80,110,0.6050,1,1,20.000000,30,18.150,600.0,0.375000


In [20]:
weapon_data_targets.head()

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,...,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit,target,target_damage,turn_target_damage_max,turn_target_damage_expected,target_damage_per_time_unit
0,pistol,1,pistol_clip,26,armor_piercing,12.0,snapshot,18,60,0.3300,...,2.400000,130,42.900,312.0,1.444444,terrain,26.0,130.0,42.900,1.444444
1,pistol,1,pistol_clip,26,armor_piercing,12.0,aimed,30,78,0.4290,...,4.000000,78,33.462,312.0,0.866667,terrain,26.0,78.0,33.462,0.866667
2,rifle,2,rifle_clip,30,armor_piercing,20.0,auto,35,35,0.1925,...,3.333333,180,34.650,600.0,0.857143,terrain,30.0,180.0,34.650,0.857143
3,rifle,2,rifle_clip,30,armor_piercing,20.0,snapshot,25,60,0.3300,...,5.000000,120,39.600,600.0,1.200000,terrain,30.0,120.0,39.600,1.200000
4,rifle,2,rifle_clip,30,armor_piercing,20.0,aimed,80,110,0.6050,...,20.000000,30,18.150,600.0,0.375000,terrain,30.0,30.0,18.150,0.375000


In [21]:
weapon_data_aliens.head()

,alien,time_units,stamina,health,bravery,reactions,firing_accuracy,throwing_accuracy,strength,psionic_strength,...,chance_of_hit_and_penetrate,penetrating_damage,turn_penetrating_damage_max,turn_penetrating_damage_expected,clip_penetrating_damage,hits_to_kill,turn_expected_kills,overkill,penetrating_damage_per_time_unit,overkill_per_time_unit
0,Floater Soldier,50,90,35,80,50,50,58,40,35,...,0.276058,18.194712,90.973558,30.021274,218.336538,2,2.5,1.389423,1.010817,0.077190
1,Floater Soldier,50,90,35,80,50,50,58,40,35,...,0.358875,18.194712,54.584135,23.416594,218.336538,2,1.5,1.389423,0.606490,0.046314
2,Floater Soldier,50,90,35,80,50,50,58,40,35,...,0.165229,22.102083,132.612500,25.527906,442.041667,2,3.0,9.204167,0.631488,0.262976
3,Floater Soldier,50,90,35,80,50,50,58,40,35,...,0.283250,22.102083,88.408333,29.174750,442.041667,2,2.0,9.204167,0.884083,0.368167
4,Floater Soldier,50,90,35,80,50,50,58,40,35,...,0.519292,22.102083,22.102083,13.371760,442.041667,2,0.5,9.204167,0.276276,0.115052
